In [1]:
import pandas as pd
import numpy as np
import keras.backend as K
from numpy import mean
from numpy import std
from keras.metrics import binary_accuracy
from sklearn.metrics import roc_auc_score, fbeta_score, recall_score, precision_score, accuracy_score

Using TensorFlow backend.


In [2]:
def fbeta2(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [3]:
t = pd.read_csv('C:/Users/jesse/Documents/FAIMS_ML/RFC_ecoli_true.txt', index_col=1)
t.drop(t.columns[0],axis=1, inplace=True)
t.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
SeqCharge,,,,,,,,,,,,,,,,
3QSVEADINGLRR,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0
2ANELLINVK,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2IGDYAGIK,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0
2VVGLSTLPEIYEK,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2QQIGVVGMAVMGR,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0


In [4]:
l = pd.read_csv('C:/Users/jesse/Documents/FAIMS_ML/LSTM_ecoli_probs.txt', index_col=0)
l.head()

,X20,X25,X30,X35,X40,X45,X50,X55,X60,X65,X70,X75,X80,X85,X90,X95
SeqCharge,,,,,,,,,,,,,,,,
3QSVEADINGLRR,0.000023,0.000027,0.000066,0.000626,0.007755,0.042606,0.180728,0.503301,0.631663,0.434623,0.315050,0.095407,0.020012,0.005538,0.001650,0.000417
2ANELLINVK,0.000021,0.000032,0.000860,0.021070,0.222184,0.702809,0.772229,0.416398,0.075275,0.007161,0.003431,0.001031,0.000466,0.000208,0.000102,0.000072
2IGDYAGIK,0.000060,0.000051,0.000079,0.000415,0.002616,0.010766,0.029524,0.104359,0.311669,0.428906,0.671399,0.554618,0.279961,0.095256,0.029222,0.006126
2VVGLSTLPEIYEK,0.000326,0.001159,0.035560,0.540519,0.832526,0.388015,0.026892,0.003401,0.000907,0.000650,0.000797,0.000488,0.000480,0.000285,0.000169,0.000162
2QQIGVVGMAVMGR,0.000015,0.000024,0.000610,0.015377,0.166626,0.652072,0.794639,0.518433,0.115050,0.009865,0.004060,0.001082,0.000426,0.000174,0.000083,0.000056


In [5]:
r = pd.read_csv('C:/Users/jesse/Documents/FAIMS_ML/RFC_ecoli_prob_predictions.txt', index_col=1)
r.drop(r.columns[0], axis=1, inplace=True)
r.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
SeqCharge,,,,,,,,,,,,,,,,
3QSVEADINGLRR,0.0,0.001874,0.001556,0.015504,0.047256,0.156251,0.354303,0.602587,0.760976,0.750509,0.662253,0.623489,0.472212,0.324393,0.127044,0.099486
2ANELLINVK,0.0,0.001131,0.004157,0.032176,0.195169,0.592096,0.810161,0.835179,0.842562,0.737052,0.437943,0.274867,0.114634,0.044136,0.032815,0.024575
2IGDYAGIK,0.0,0.000000,0.007035,0.026984,0.053899,0.334681,0.347333,0.504760,0.618234,0.755630,0.785818,0.800131,0.802851,0.692346,0.589119,0.443617
2VVGLSTLPEIYEK,0.0,0.016411,0.220997,0.770720,0.849681,0.741074,0.311493,0.149818,0.027580,0.003395,0.001060,0.000000,0.008100,0.000000,0.000000,0.002125
2QQIGVVGMAVMGR,0.0,0.014895,0.153543,0.513586,0.683072,0.787122,0.717835,0.364294,0.215734,0.147760,0.070552,0.022806,0.019432,0.014916,0.004306,0.000000


# Random forest metric

In [6]:
## compute metrics
print('RF metrics')
m = binary_accuracy(np.asarray(t), np.asarray(r))
print('binary accuracy =', np.round(m.numpy().sum()/m.shape[0], 4))
print('precision =', np.round(precision_score(t, np.round(r), average='micro'),4) )
print('recall =', np.round(recall_score(t, np.round(r), average='micro'),4) )
print('AUROC =', np.round(roc_auc_score(t, r, average='micro'),4) )
print('f2 score =', np.round(fbeta_score(t, np.round(r), beta=2, average='micro'),4))
matchsum = np.sum(np.asarray(t*np.asarray(np.round(r))), axis=1)
print('at least one correct =', np.round((matchsum>0).sum()/matchsum.shape[0],4))

RF metrics
binary accuracy = 0.8022
precision = 0.3789
recall = 0.8084
AUROC = 0.8834
f2 score = 0.659
at least one correct = 0.8851


# LSTM metrics

In [7]:
## compute metrics
print( 'LSTM metrics')
m = binary_accuracy(np.asarray(t), np.asarray(l))
print('binary accuracy', np.round(m.numpy().sum()/m.shape[0], 4))
print('precision = ', np.round(precision_score(t, np.round(l), average='micro'), 4))
print('recall = ', np.round(recall_score(t, np.round(l), average='micro'),5))
print('AUROC = ', np.round(roc_auc_score(t, l, average='micro'),4))
print('F2 score = ', np.round(fbeta_score(t, np.round(l), beta=2, average='micro'),4))
matchsum = np.sum(np.asarray(t*np.asarray(np.round(l))), axis=1)
one_right_tmp = (matchsum>0).sum()/matchsum.shape[0]
print('at least one correct match, LSTM', np.round(one_right_tmp, 4))

LSTM metrics
binary accuracy 0.9027
precision =  0.6504
recall =  0.5487
AUROC =  0.9369
F2 score =  0.5664
at least one correct match, LSTM 0.7447


# Ensemble (average) metrics

In [8]:
both = (np.asarray(l)+np.asarray(r))/2
## compute metrics
print('Ensemble (average) metrics')
### binary accuracy ensemble
m = binary_accuracy(np.asarray(t), both, threshold=0.5)

print('binary accuracy =', np.round(m.numpy().sum()/m.shape[0], 4))
print('precision =', np.round(precision_score(t, np.round(both), average='micro'),4))
print('recall =', np.round(recall_score(t, np.round(both), average='micro'),4) )
print('auroc =', np.round(roc_auc_score(t, both, average='micro'),4))
print('f2 score =', np.round(fbeta_score(t, np.round(both), beta=2, average='micro'),4))
matchsum = np.sum(np.asarray(t*np.asarray(np.round(both))), axis=1)
print('at least one correct =', np.round((matchsum>0).sum()/matchsum.shape[0],4))

Ensemble (average) metrics
binary accuracy = 0.895
precision = 0.583
recall = 0.6837
auroc = 0.928
f2 score = 0.6609
at least one correct = 0.8195


In [9]:
# at least one wrong - RF
np.any(np.asarray(t-np.asarray(np.round(r)))<0, axis=1).sum()/r.shape[0]

0.9406351453609002

In [10]:
# at least one wrong -LSTM
np.any(np.asarray(t-np.asarray(np.round(l)))<0, axis=1).sum()/r.shape[0]

0.4767495789293822

In [11]:
# at least one wrong - Ensemble
np.any(np.asarray(t-np.asarray(np.round(both)))<0, axis=1).sum()/r.shape[0]

0.6618497815314766